#### **Inflation Forecasting: Data Processing**

**Author: Fabián Villarreal Sosa**

Notebook to compare between machine learning models and classic econometric models to forecast inflation in Ecuador.

In [21]:
# Librerias
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import mplcursors
import seaborn as sns

import datetime as dt

In [22]:
# Change directory
os.chdir('C:/Users/fvill/Dropbox/DS_ML_Projects/Inflation_Forecasting/Data')
os.getcwd()

'C:\\Users\\fvill\\Dropbox\\DS_ML_Projects\\Inflation_Forecasting\\Data'

##### **IPC**

Data source: [INEC](https://www.ecuadorencifras.gob.ec//indice-de-precios-al-consumidor/)

In [23]:
# Data
df = pd.read_csv('1. INDICE.csv', skiprows = 1, encoding = 'ISO-8859-1')

# Data treatment
df = df.iloc[1:]
df = df[df['Enero'].notna()]

for col in df.columns:
  df[col] = df[col].astype(float)

df = df.rename(columns = {'AÑOS': 'year'})

# Reshape
df.columns = ['year'] + list(range(1,13))

df = pd.melt(df,
              id_vars = ['year'],
              var_name = 'month',
              value_vars = df.drop(['year'], axis = 1).columns,
              value_name = 'ipc')

df.sort_values(['year', 'month'], inplace = True, ignore_index = True)

df = df[df['ipc'].notna()]

# Date
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_ipc = df
df_ipc.head()

,date,ipc
0,1969-01-01,0.01
1,1969-02-01,0.02
2,1969-03-01,0.02
3,1969-04-01,0.02
4,1969-05-01,0.02


##### **IAE-NP**

Data source: [SRI](https://www.sri.gob.ec/serie-historica-mensual-actividad-empresarial-no-petrolera)

In [24]:
# Data
df = pd.read_excel('IAEN2109.xlsx', sheet_name = '1.Serie mensual', skiprows = 4)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['date', 'ind_man', 'ind_com', 'ind_con', 'ind_ser', 'iae_np']
df = df[df['iae_np'].notna()]

df['date'] = pd.to_datetime(df['date'])

# Dataframe
df.sort_values(['date'])
df_iae_np = df
df_iae_np.head()

,date,ind_man,ind_com,ind_con,ind_ser,iae_np
0,2003-01-01,100.000000,100.000000,100.000000,100.000000,100.000000
1,2003-02-01,99.489992,100.296631,100.012682,100.569706,100.080926
2,2003-03-01,98.254307,98.062861,97.899008,100.298454,98.588371
3,2003-04-01,98.937460,99.945706,97.589871,104.152288,99.994108
4,2003-05-01,101.657028,100.805479,96.960736,103.750605,100.588182


##### **IDEAC**

Data source: [BCE](https://contenido.bce.fin.ec/documentos/Administracion/bi_IDEAC.html)

In [25]:
# Data
df = pd.read_excel('Índice_de_actividad_económica_coyuntural_-_Mensual.xlsx', sheet_name = 'Indicador', skiprows = 5)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['year', 'mes', 'ideac_ct', 'ideac_sb']
df = df[df['ideac_sb'].notna()]


mes = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
df['month'] = 0
counter = 1
for m in mes:
  df.loc[df['mes'] == m, 'month'] = counter
  counter = counter + 1 

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))

df = df.drop(['year', 'month', 'mes'], axis = 1)
df = df[['date', 'ideac_ct', 'ideac_sb']]

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_ideac = df
df_ideac.head()

C:\Users\fvill\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,ideac_ct,ideac_sb
0,1995-01-01,54.910278,52.863964
1,1995-02-01,55.203971,51.415762
2,1995-03-01,55.660572,58.448572
3,1995-04-01,56.252675,56.128569
4,1995-05-01,56.955838,54.810635


##### **Money Supply and Total Liquidity**

Data source: [BCE](https://contenido.bce.fin.ec/documentos/Administracion/bi_menuBoletin.html#)

In [26]:
# Data
df = pd.read_excel('Oferta_Monetaria_(M1)_y_Liquidez_Total_(M2)_-_Mensual.xlsx', sheet_name = 'Indicador', skiprows = 5)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['year', 'mes', 'base_mon', 'caja_bce', 'casa_osd', 'cuasidinero', 
              'dep_vista', 'din_elec', 'esp_mon_cir', 'liq_m2', 'mon_frac',
              'mult_m1_bm', 'mult_m2_bm', 'ofer_m1', 'res_ban']
df = df[df['liq_m2'].notna()]

mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['month'] = 0
counter = 1
for m in mes:
  df.loc[df['mes'] == m, 'month'] = counter
  counter = counter + 1 

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month', 'mes'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

df['din_elec'] = df['din_elec'].replace('-', np.nan)
df['din_elec'] = pd.to_numeric(df['din_elec'])

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_msupply = df
df_msupply.head()

,date,base_mon,caja_bce,casa_osd,cuasidinero,dep_vista,din_elec,esp_mon_cir,liq_m2,mon_frac,mult_m1_bm,mult_m2_bm,ofer_m1,res_ban
0,2000-01-01,873.37,82.04,96.54,2208.43,533.76,NaN,536.23,3278.42,0.0,1.225128,3.753758,1069.99,158.56
1,2000-02-01,839.39,98.85,85.54,2225.48,596.49,NaN,530.42,3352.39,0.0,1.342534,3.993841,1126.91,124.58
2,2000-03-01,921.00,54.78,102.57,2199.92,696.03,NaN,557.46,3453.41,0.0,1.361010,3.749631,1253.49,206.19
3,2000-04-01,1002.99,96.78,114.36,2380.72,697.88,NaN,567.96,3646.56,0.0,1.262066,3.635689,1265.84,223.89
4,2000-05-01,1057.62,104.53,125.77,2433.46,801.82,NaN,625.34,3860.62,0.0,1.349407,3.650290,1427.16,201.98


##### **Commodity Prices**

Data source: [FMI](https://data.imf.org/?sk=471DDDF8-D8A7-499A-81BA-5B332C01F8B9&sId=1390030341854)

In [27]:
# Data
df = pd.read_excel('Primary_Commodity_Price_System_PCPS.xlsx', sheet_name = 'Data')

# Data treatment
df = df.drop(['Unnamed: 1'], axis = 1)

df = df.transpose()
df.columns = df.iloc[0,]
df = df.drop(df.index[0])
df.columns.name = None
df = df.reset_index()

df = df.iloc[:, : -2]
df.columns = ['fecha', 'agr_raw_mat', 'apsp_crude', 'coal', 'energy', 'fertil', 'food_bever', 'ind_materials', 'metal', 'natural_gas', 'non-fuel', 'precious_metals']

df['date'] = pd.date_range('1990-1', freq = 'M', periods = len(df))

df = df.drop(['fecha'], axis = 1)
col = df.pop('date')
df.insert(0, col.name, col)

for col in df.drop(['date'], axis = 1).columns:
  df[col] = df[col].astype(float)

df['date'] = df['date'] - pd.offsets.MonthBegin(1)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_commod = df
df_commod.tail()

,date,agr_raw_mat,apsp_crude,coal,energy,fertil,food_bever,ind_materials,metal,natural_gas,non-fuel,precious_metals
380,2021-09-01,108.462567,166.180587,260.607197,213.711309,195.277985,128.339385,167.860409,191.469690,345.327276,144.297855,145.543701
381,2021-10-01,112.829024,188.475001,338.060977,257.372357,225.753607,130.512432,172.158216,195.740209,458.465347,147.460358,145.241816
382,2021-11-01,116.294941,184.639030,235.572486,237.354472,250.892263,131.913587,161.528058,179.507185,418.739275,146.843471,148.661804
383,2021-12-01,114.319145,169.409225,236.176016,241.339613,251.173837,134.675306,168.617548,190.199919,506.703151,149.325023,144.551996
384,2022-01-01,117.708599,194.548961,298.621217,242.756336,223.693276,138.960941,182.846230,208.736944,392.143667,155.049161,147.999149


##### **WTI Spot Price**

Data source: [EIA](https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=RWTC&f=M)

In [28]:
# Data
df = pd.read_excel('RWTCm.xls', sheet_name = 'Data 1', skiprows = 2)

# Data treatment
df.columns = ['date', 'wti_price']
df['date'] = df['date'] - pd.offsets.MonthBegin(1)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_wti = df
df_wti.head()

,date,wti_price
0,1986-01-01,22.93
1,1986-02-01,15.46
2,1986-03-01,12.61
3,1986-04-01,12.84
4,1986-05-01,15.38


##### **GDP**

Data source: [BCE](https://contenido.bce.fin.ec/documentos/Administracion/bi_menuCNTdef.html#)

In [29]:
# Data
df = pd.read_excel('PUB_-_PIB_Trimestral.xlsx', sheet_name = 'PIB Trimestral', skiprows =  9)

# Data treatment
df = df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 8'], axis = 1)
df.columns = ['year', 'trimestre', 'pib_nom_USDK', 'pib_real_USDK', 'pib_real_tt4', 'pib_real_tt1']
df = df.drop(['pib_real_tt4', 'pib_real_tt1'], axis = 1)
df = df[df['pib_nom_USDK'].notna()]

df['year'].fillna(method = 'ffill', inplace = True)

df['date'] = pd.date_range('2000-1', freq = 'Q', periods = len(df))

# Interpolate
df['date'] = pd.to_datetime(df['date']).dt.to_period('M')
df = df.set_index('date').resample('M').interpolate()
df = df.reset_index()
df['date'] = df['date'].dt.to_timestamp('s')
df = df.drop(['year', 'trimestre'], axis = 1)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_gpd = df
df_gpd.head()

,date,pib_nom_USDK,pib_real_USDK
0,2000-03-01,3819126.0,9.134587e+06
1,2000-04-01,4013577.0,9.196529e+06
2,2000-05-01,4208028.0,9.258472e+06
3,2000-06-01,4402479.0,9.320414e+06
4,2000-07-01,4570537.0,9.396440e+06


##### **Interest Rates**

Data source: [BCE](https://contenido.bce.fin.ec/documentos/Administracion/bi_menuTasas.html)

In [30]:
# Data
df = pd.read_excel('Tasas_de_interés_referenciales_vigentes_-_Mensual.xlsx', sheet_name = 'Indicador', skiprows = 5)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['year', 'mes', 'loan_rate', 'deposit_rate']
df = df[df['loan_rate'].notna()]

mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['month'] = 0
counter = 1
for m in mes:
  df.loc[df['mes'] == m, 'month'] = counter
  counter = counter + 1 

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month', 'mes'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_irates = df
df_irates.head()

C:\Users\fvill\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,loan_rate,deposit_rate
0,2000-04-01,67.56,36.83
1,2000-05-01,75.75,42.90
2,2000-06-01,62.03,33.38
3,2000-07-01,75.85,38.50
4,2000-08-01,65.36,31.82


##### **Credit to the Private Sector**

Data source: [BCE](https://sintesis.bce.fin.ec/BOE/OpenDocument/2109181649/OpenDocument/opendoc/openDocument.faces?logonSuccessful=true&shareId=2)

In [31]:
# Data
df = pd.read_excel('Crédito_al_sector_privado.xlsx', sheet_name = 'Indicador', skiprows = 7)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['year', 'mes', 'credit_priv_USDM']
df = df[df['credit_priv_USDM'].notna()]

mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['month'] = 0
counter = 1
for m in mes:
  df.loc[df['mes'] == m, 'month'] = counter
  counter = counter + 1 

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month', 'mes'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_credit = df
df_credit.head()

C:\Users\fvill\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,credit_priv_USDM
0,2003-02-01,4149.011068
1,2003-03-01,4158.879078
2,2003-04-01,4174.240000
3,2003-05-01,4279.520000
4,2003-06-01,4282.840000


##### **FRED Data**

Data source: 

* [Consumer Price Index for All Urban Consumers: All Items in U.S. City Average](https://fred.stlouisfed.org/series/CPIAUCSL)
* [Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity](https://fred.stlouisfed.org/series/DGS10)

In [32]:
from fredapi import Fred
fred = Fred(api_key = '665265cf43b59a471710de63b8503e66')

In [33]:
# Data: Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity
df = {}
df['TNX'] = fred.get_series('DGS10')
df = pd.DataFrame(df)

# Data treatment
df = df.reset_index()
df = df.rename(columns = {'index': 'date'})
df = df.groupby(pd.PeriodIndex(df['date'], freq = 'M'))[['TNX']].mean().reset_index()

df['date'] = df['date'].dt.to_timestamp('s')

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_tnx = df
df_tnx.head()

,date,TNX
0,1962-01-01,4.083182
1,1962-02-01,4.039444
2,1962-03-01,3.930455
3,1962-04-01,3.843000
4,1962-05-01,3.873636


In [34]:
# Data: Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
df = {}
df['CPI_US'] = fred.get_series('CPIAUCSL')
df = pd.DataFrame(df)

# Data treatment
df = df.reset_index()
df = df.rename(columns = {'index': 'date'})

# Data treatment
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_cpi_us = df
df_cpi_us.head()

,date,CPI_US
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [35]:
# Merge data
df = pd.merge(df_cpi_us, df_tnx, how = 'left', on = 'date')

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_fred = df
df_fred.head()

,date,CPI_US,TNX
0,1947-01-01,21.48,NaN
1,1947-02-01,21.62,NaN
2,1947-03-01,22.00,NaN
3,1947-04-01,22.00,NaN
4,1947-05-01,21.95,NaN


##### **International Reserves**

Data source: [BCE](https://contenido.bce.fin.ec/documentos/Administracion/bi_menuReservas.html)

In [36]:
# Data
df = pd.read_excel('Reservas_Internacionales_-_Mensual.xlsx', sheet_name = 'Indicador', skiprows = 5)

# Data treatment
df = df.drop(['Unnamed: 0'], axis = 1)
df.columns = ['year', 'mes', 'pos_divisas', 'oro_monetario', 'DEGs', 'pos_reserva_FMI', 'pos_ALADI', 'pos_sucre', 'res_internacional']
df = df[df['res_internacional'].notna()]

mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['month'] = 0
counter = 1
for m in mes:
  df.loc[df['mes'] == m, 'month'] = counter
  counter = counter + 1

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month', 'mes'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Replace characters
df['pos_sucre'] = df['pos_sucre'].replace('-', np.nan)

cols = df.drop(['date'], axis = 1).columns
for col in cols:
  df[col] = df[col].replace('\.', '', regex = True)
  df[col] = df[col].replace(',', '.', regex = True)
  df[col] = pd.to_numeric(df[col])

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_res = df
df_res.head()

C:\Users\fvill\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,pos_divisas,oro_monetario,DEGs,pos_reserva_FMI,pos_ALADI,pos_sucre,res_internacional
0,2000-01-01,605.26,239.50,2.44,23.32,-17.90,NaN,852.62
1,2000-02-01,645.82,248.40,2.10,23.07,-9.63,NaN,909.76
2,2000-03-01,529.07,233.43,2.09,23.05,-8.78,NaN,778.86
3,2000-04-01,691.66,233.59,1.31,22.63,-10.59,NaN,938.60
4,2000-05-01,648.14,231.18,0.89,22.72,-12.14,NaN,890.79


##### **Producer Price Index**

Data source: [INEC](https://www.ecuadorencifras.gob.ec//indice-de-precios-al-productor-de-disponibilidad-nacional/)

In [37]:
# Data
df = pd.read_csv('1. Indice_General_IPP.csv', skiprows = 6, encoding = 'ISO-8859-1')

# Data treatment
df = df.drop(['Unnamed: 13', 'Unnamed: 14'], axis = 1)
df = df[df['Enero'].notna()]

for col in df.columns:
  df[col] = df[col].astype(float)

df = df.rename(columns = {'Años': 'year'})

# Reshape
df.columns = ['year'] + list(range(1,13))

df = pd.melt(df,
              id_vars = ['year'],
              var_name = 'month',
              value_vars = df.drop(['year'], axis = 1).columns,
              value_name = 'ipp')

df.sort_values(['year', 'month'], inplace = True, ignore_index = True)

df = df[df['ipp'].notna()]

# Date
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_ipp = df
df_ipp.head()

,date,ipp
0,1998-01-01,11.11
1,1998-02-01,11.38
2,1998-03-01,11.68
3,1998-04-01,12.46
4,1998-05-01,12.67


##### **Construction Price Index**

Data source: [INEC](https://www.ecuadorencifras.gob.ec//indice-de-precios-de-la-construccion/)

In [38]:
# Data
df = pd.read_csv('1. Indice_IPCons.csv', skiprows = 4, encoding = 'ISO-8859-1')

# Data treatment
df = df.iloc[:, 0:13]
df = df[df['ENERO'].notna()]

for col in df.columns:
  df[col] = df[col].replace('\*', '', regex = True)

for col in df.columns:
  df[col] = df[col].astype(float)

df = df.rename(columns = {'AÑOS': 'year'})

# Reshape
df.columns = ['year'] + list(range(1,13))

df = pd.melt(df,
              id_vars = ['year'],
              var_name = 'month',
              value_vars = df.drop(['year'], axis = 1).columns,
              value_name = 'ipcons')

df.sort_values(['year', 'month'], inplace = True, ignore_index = True)

df = df[df['ipcons'].notna()]

# Date
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY = 1))
df = df.drop(['year', 'month'], axis = 1)

col = df.pop('date')
df.insert(0, col.name, col)

# Dataframe
df.sort_values(['date'], ascending = True, ignore_index = True, inplace = True)
df_ipcons = df
df_ipcons.head()

,date,ipcons
0,1984-01-01,0.27
1,1984-02-01,0.28
2,1984-03-01,0.30
3,1984-04-01,0.30
4,1984-05-01,0.31


##### **Final DataFrame**

In [39]:
from functools import reduce

# Merge
dataframes = [df_iae_np, df_ideac, df_msupply, df_commod, df_wti, df_gpd, df_irates, df_credit, df_fred, df_res, df_ipp, df_ipcons]

df = df_ipc
for data in dataframes:
  df = pd.merge(df, data, how = 'left', on = 'date')

# Time filter
f1 = df['date'] >= '2004-01-01'
f2 = df['date'] <= '2021-09-01'
df = df.loc[f1 & f2, :]

In [40]:
# Save as csv
df.to_csv('final_dataframe_ipc.csv', index = False)